# Nested Logit Model: Compute Market Shares

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.optimization as opt
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable
import seaborn as sns
import matplotlib.pyplot as plt

**Import Optima data**

In [2]:
pandas = pd.read_csv("../../Data/8-DiscreteChoiceModels/optima.dat",sep='\t')
database = db.Database ("data/optima", pandas)

**Use collumn names as variables**

In [3]:
globals().update(database.variables)

**Exclude some unwanted entries**

In [4]:
exclude = (Choice == -1.)
database.remove(exclude)

**Define some dummy variables**

In [5]:
male = (Gender == 1)
female = (Gender == 2)
unreportedGender = (Gender == -1)

fulltime = (OccupStat == 1)
notfulltime = (OccupStat != 1)

**Rescale some data**

In [6]:
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10 
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5

**Create parameters to be estimated**

In [7]:
ASC_CAR              = Beta('ASC_CAR',0,None,None,0)
ASC_PT               = Beta('ASC_PT',0,None,None,1)
ASC_SM               = Beta('ASC_SM',0,None,None,0)
BETA_TIME_FULLTIME   = Beta('BETA_TIME_FULLTIME',0,None,None,0)
BETA_TIME_OTHER      = Beta('BETA_TIME_OTHER',0,None,None,0)
BETA_DIST_MALE       = Beta('BETA_DIST_MALE',0,None,None,0)
BETA_DIST_FEMALE     = Beta('BETA_DIST_FEMALE',0,None,None,0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED',0,None,None,0)
BETA_COST            = Beta('BETA_COST',0,None,None,0)

**Define the utility functions**

\begin{align}
V_{PT} & = \beta_{PT} + \beta_{time_{fulltime}} X_{time_{PT}} X_{fulltime} + \beta_{time_{other}} X_{time_{PT}} X_{not\_fulltime} + \beta_{cost} X_{cost_{PT}} \\
V_{car} & = \beta_{car} + \beta_{time_{fulltime}} X_{time_{car}} X_{fulltime} + \beta_{time_{other}} X_{time_{car}} X_{not\_fulltime} + \beta_{cost} X_{cost_{car}} \\
V_{SM} & = \beta_{SM} + \beta_{male} X_{distance} X_{male} + \beta_{female} X_{distance} X_{female} + \beta_{unreported} X_{distance} X_{unreported}
\end{align}

In [8]:
V_PT = ASC_PT + BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
       BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
       BETA_COST * MarginalCostPT_scaled
V_CAR = ASC_CAR + \
        BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
        BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
        BETA_COST * CostCarCHF_scaled
V_SM = ASC_SM + \
       BETA_DIST_MALE * distance_km_scaled * male + \
       BETA_DIST_FEMALE * distance_km_scaled * female + \
       BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

**Associate utility functions with alternatives and associate availability of alternatives**

In this example all alternatives are available for each individual

In [9]:
V = {0: V_PT,
     1: V_CAR,
     2: V_SM}

av = {0: 1,
      1: 1,
      2: 1}

**Define the nests**

1. Define the nests paramenters
2. List alternatives in nests

In [10]:
MU_NO_CAR = Beta('MU_NO_CAR', 1.,1.,None,0)

CAR_NEST = 1., [1]
NO_CAR_NEST = MU_NO_CAR, [0, 2]

nests = CAR_NEST, NO_CAR_NEST

**Define the choice probabilities**

In [11]:
prob_pt  = models.nested(V, av , nests , 0)
prob_car = models.nested(V, av , nests , 1)
prob_sm  = models.nested(V, av , nests , 2)

**Compute normalizing weights for each alternative**

In [12]:
sumWeight = database.data['Weight'].sum()
normalized_Weight = Weight * len(database.data['Weight']) / sumWeight

**Define what we want to simulate**

1. Normalized weights
2. Choice probabilities for each choice
3. Revenues for the Public Transportation alternative

In [13]:
simulate ={'weight':  normalized_Weight ,
           'Prob. car':  prob_car ,
           'Prob. public transportation':  prob_pt ,
           'Prob. slow modes': prob_sm ,
           'Revenue public transportation': prob_pt * MarginalCostPT}

**Define the Biogeme object**

In [14]:
biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "optima_nested_logit_market"

**Retrieve the names of the variables we want to use. Then retrieve the results from the model that we estimated earlier**

In [15]:
betas = biogeme.freeBetaNames

print('Extracting the following variables:')
for k in betas:
    print('\t',k)

results = res.bioResults(pickleFile='optima_nested_logit.pickle')
betaValues = results.getBetaValues ()

Extracting the following variables:
	 ASC_CAR
	 ASC_SM
	 BETA_COST
	 BETA_DIST_FEMALE
	 BETA_DIST_MALE
	 BETA_DIST_UNREPORTED
	 BETA_TIME_FULLTIME
	 BETA_TIME_OTHER
	 MU_NO_CAR


**Perform the simulation**

In [16]:
simulatedValues = biogeme.simulate(betaValues)

**Compute confidente intervals using this simulation. Compare results using normal estimation and with bootstrapping.**

In [17]:
betas = biogeme.freeBetaNames
b_bootstrap = results.getBetasForSensitivityAnalysis(betas)
b_normal = results.getBetasForSensitivityAnalysis(
    betas, size=100, useBootstrap=False
)

In [19]:
# Returns data frame containing, for each simulated value, the left
# and right bounds of the confidence interval calculated by
# simulation.
left_bootstrap, right_bootstrap = biogeme.confidenceIntervals(b_bootstrap, 0.9)
left_normal, right_normal = biogeme.confidenceIntervals(b_normal, 0.9)

**Car Market Share**

In [21]:
simulatedValues['Weighted prob. car'] = (
    simulatedValues['weight'] * simulatedValues['Prob. car']
)
left_bootstrap['Weighted prob. car'] = (
    left_bootstrap['weight'] * left_bootstrap['Prob. car']
)
right_bootstrap['Weighted prob. car'] = (
    right_bootstrap['weight'] * right_bootstrap['Prob. car']
)
left_normal['Weighted prob. car'] = (
    left_normal['weight'] * left_normal['Prob. car']
)
right_normal['Weighted prob. car'] = (
    right_normal['weight'] * right_normal['Prob. car']
)

marketShare_car = simulatedValues['Weighted prob. car'].mean()

marketShare_car_left_bootstrap = left_bootstrap['Weighted prob. car'].mean()
marketShare_car_right_bootstrap = right_bootstrap['Weighted prob. car'].mean()
marketShare_car_left_normal = left_normal['Weighted prob. car'].mean()
marketShare_car_right_normal = right_normal['Weighted prob. car'].mean()

print(
    f'Market share for car: {100*marketShare_car:.1f}% '
    f'bootstrap[{100*marketShare_car_left_bootstrap:.1f}%, '
    f'{100*marketShare_car_right_bootstrap:.1f}%]'
    f' normal[{100*marketShare_car_left_normal:.1f}%, '
    f'{100*marketShare_car_right_normal:.1f}%]'
)

Market share for car: 65.3% bootstrap[61.5%, 69.2%] normal[59.9%, 68.8%]


**Public Transportation Market Share**

In [22]:

simulatedValues['Weighted prob. PT'] = (
    simulatedValues['weight'] * simulatedValues['Prob. public transportation']
)

marketShare_pt = simulatedValues['Weighted prob. PT'].mean()

marketShare_pt_left_bootstrap = (
    left_bootstrap['Prob. public transportation'] * left_bootstrap['weight']
).mean()
marketShare_pt_right_bootstrap = (
    right_bootstrap['Prob. public transportation'] * right_bootstrap['weight']
).mean()
marketShare_pt_left_normal = (
    left_normal['Prob. public transportation'] * left_normal['weight']
).mean()
marketShare_pt_right_normal = (
    right_normal['Prob. public transportation'] * right_normal['weight']
).mean()

print(
    f'Market share for PT: {100*marketShare_pt:.1f}% '
    f'bootstrap[{100*marketShare_pt_left_bootstrap:.1f}%, '
    f'{100*marketShare_pt_right_bootstrap:.1f}%]'
    f'normal[{100*marketShare_pt_left_normal:.1f}%, '
    f'{100*marketShare_pt_right_normal:.1f}%]'
)


Market share for PT: 28.1% bootstrap[24.1%, 31.8%]normal[23.7%, 32.5%]


**Slow Modes Market Share**

In [23]:
marketShare_sm = (
    simulatedValues['Prob. slow modes'] * simulatedValues['weight']
).mean()

marketShare_sm_left_bootstrap = (
    left_bootstrap['Prob. slow modes'] * left_bootstrap['weight']
).mean()
marketShare_sm_right_bootstrap = (
    right_bootstrap['Prob. slow modes'] * right_bootstrap['weight']
).mean()
marketShare_sm_left_normal = (
    left_normal['Prob. slow modes'] * left_normal['weight']
).mean()
marketShare_sm_right_normal = (
    right_normal['Prob. slow modes'] * right_normal['weight']
).mean()

print(
    f'Market share for slow modes: {100*marketShare_sm:.1f}% '
    f'bootstrap[{100*marketShare_sm_left_bootstrap:.1f}%, '
    f'{100*marketShare_sm_right_bootstrap:.1f}%]'
    f' normal[{100*marketShare_sm_left_normal:.1f}%, '
    f'{100*marketShare_sm_right_normal:.1f}%]'
)


Market share for slow modes: 6.6% bootstrap[4.2%, 9.5%] normal[4.9%, 11.0%]


**Public Transportation revenue estimation**

In [24]:

revenues_pt = (
    simulatedValues['Revenue public transportation']
    * simulatedValues['weight']
).sum()

revenues_pt_left_bootstrap = (
    left_bootstrap['Revenue public transportation'] * left_bootstrap['weight']
).sum()
revenues_pt_right_bootstrap = (
    right_bootstrap['Revenue public transportation']
    * right_bootstrap['weight']
).sum()
revenues_pt_left_normal = (
    left_normal['Revenue public transportation'] * left_normal['weight']
).sum()
revenues_pt_right_normal = (
    right_normal['Revenue public transportation'] * right_normal['weight']
).sum()
print(
    f'Revenues for PT: {revenues_pt:.3f} '
    f'bootstrap[{revenues_pt_left_bootstrap:.3f}, '
    f'{revenues_pt_right_bootstrap:.3f}]'
    f' normal[{revenues_pt_left_normal:.3f}, '
    f'{revenues_pt_right_normal:.3f}]'
)

Revenues for PT: 3018.371 bootstrap[2448.232, 3579.859] normal[2485.135, 3875.832]
